# ML testing: experiment #1

This notebook involves testing for the MRI conference abstract. This notebook shows TOP based models

### import libraries

In [1]:
import os       # using operating system dependent functionality (folders)
import sys

import glob

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# demo stuff
#import ipywidgets as widgets
#import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

### import data

In [2]:
filepath_mri = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath_mri,'StrokeMRI_pvc2c.csv') 

filepath_top = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top = os.path.join(filepath_top,'TOP_pvc2c.csv') 

In [3]:
TOP = pd.read_csv(filename_top)
StrokeMRI = pd.read_csv(filename_mri)

In [4]:
TOP = TOP.drop(TOP.columns[0],axis=1)
#TOP

In [5]:
StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
StrokeMRI

,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
0,sub-59082_1_ses-1_run-1,43.172603,F,0.64754,0.49441,0.31320,0.44500,0.44500,1.249,24.0,8.0434,1.7273,1.9776,1.7141,1.9140,26.8262,104.3155,90.5047,73.8758,84.4527
1,sub-59083_1_ses-1_run-1,66.367123,F,0.60517,0.48594,0.42304,0.39968,0.39968,14.597,25.0,3.7791,1.9669,1.7154,2.6394,1.8815,24.9597,75.7051,65.3032,65.6771,67.1158
2,sub-59085_1_ses-1_run-1,55.838356,F,0.61724,0.53779,0.33692,0.41371,0.41371,6.341,30.0,5.1248,1.8146,1.9845,2.0827,1.9501,27.7117,81.0966,72.9980,60.8317,64.3183
3,sub-59086_1_ses-1_run-1,48.238356,F,0.63402,0.48542,0.29969,0.44677,0.44677,3.070,23.0,5.0993,1.6616,1.9321,2.0319,1.7644,24.9029,75.6702,70.6205,55.1641,61.8921
4,sub-59087_1_ses-1_run-1,58.616438,F,0.57348,0.45642,0.35190,0.41502,0.41502,1.761,23.0,6.6565,2.0185,1.8745,2.0016,1.9140,18.5264,63.3017,53.8047,44.1930,49.1181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,sub-59440_1_ses-1_run-1,73.928767,M,0.57259,0.52662,0.51982,0.35366,0.35366,26.949,64.0,2.2578,2.0305,2.0193,1.9228,2.0773,27.9190,82.7343,72.6401,54.3780,59.3805
510,sub-59440_2_ses-2_run-1,74.769863,M,0.57528,0.50907,0.52840,0.35671,0.35671,22.915,32.0,2.3344,2.5824,2.6416,2.2723,2.6217,28.7051,82.7827,76.7499,65.4477,61.8818
511,sub-59441_2_ses-2_run-1,74.512329,M,0.58675,0.47585,0.42899,0.39337,0.39337,5.249,30.0,3.8390,1.8317,1.8702,1.6799,1.7434,27.1356,66.0938,63.8049,52.7282,54.3541
512,sub-59442_1_ses-1_run-1,67.526027,M,0.63929,0.55235,0.39661,0.40251,0.40251,8.612,9.0,4.9303,1.6693,1.9822,2.1802,1.8634,24.0485,71.8750,65.1286,56.4845,59.2195


In [ ]:
# Now we need to flip the sex back to numbers for a correlation
sex_mapping = {'F':0,'M':1}
TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
TOP.head(3)

In [ ]:
StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
StrokeMRI.head(3)

## Build ML models

In [ ]:
ml_matrix = TOP.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

In [ ]:
linr = LinearRegression()
linr.fit(X_train, y_train)

In [ ]:
y_pred = linr.predict(X_test)

In [ ]:
print('R2 score Linear regression: %.3f' % linr.score(X_test,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
llreg = linear_model.LassoLars(alpha=0.01)
llreg.fit(X_train, y_train)

In [ ]:
y_pred = llreg.predict(X_test)
print('R2 score Lasso regression: %.3f' % llreg.score(X_test,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
dtree = tree.DecisionTreeRegressor()
dtree.fit(X_train, y_train)

In [ ]:
y_pred = dtree.predict(X_test)
print('R2 score dtree regression: %.3f' % dtree.score(X_test,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
regr = MLPRegressor(random_state=1, max_iter=900)
regr.fit(X_train, y_train)

In [ ]:
y_pred = regr.predict(X_test)

In [ ]:
print('R2 score neural network mlp regression: %.3f' % regr.score(X_test,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
eregr = ElasticNetCV(cv=5, random_state=12)
eregr.fit(X_train, y_train)

In [ ]:
y_pred = eregr.predict(X_test)
print('R2 score elasticnetcv regression: %.3f' % eregr.score(X_test,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
etreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
etreg.fit(X_train, y_train)

In [ ]:
y_pred = etreg.predict(X_test)
print('R2 score elasticnetcv regression: %.3f' % etreg.score(X_test,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
## SAve off models

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
# joblib.dump(linr, ('../result_models/'+ 'unharm_top_linr.sav'))
# joblib.dump(llreg, ('../result_models/'+ 'unharm_top_lassor.sav'))
# joblib.dump(dtree, ('../result_models/'+ 'unharm_top_dtree.sav'))
# joblib.dump(regr, ('../result_models/'+ 'unharm_top_regr.sav'))

In [ ]:
# joblib.dump(eregr, ('../result_models/'+ 'unharm_top_extratree.sav'))
# joblib.dump(etreg, ('../result_models/'+ 'unharm_top_elasticregr.sav'))

## Run models on other dataset

In [ ]:
mri_ml_matrix = StrokeMRI.drop('participant_id', axis=1)
X_mri = mri_ml_matrix.drop('age', axis =1)
X_mri = X_mri.values
X_mri = X_mri.astype('float')
y_mri = mri_ml_matrix['age'].values
y_mri=y_mri.astype('float')

In [ ]:
X_mri_test = X_mri
y_mri_test = y_mri

In [ ]:
y_mri_pred = linr.predict(X_mri_test)
print('R2 score Linear regression: %.3f' % linr.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
y_mri_pred = llreg.predict(X_mri_test)
print('R2 score Lasso regression: %.3f' % llreg.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
y_mri_pred = dtree.predict(X_mri_test)
print('R2 score Decision tree: %.3f' % dtree.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
y_mri_pred = regr.predict(X_mri_test)
print('R2 score MLP regression: %.3f' % regr.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
# plt.figure(figsize=(10,10))
# plt.scatter(y_test, y_pred, c='crimson')
# plt.yscale('log')
# plt.xscale('log')

# p1 = max(max(y_pred), max(y_test))
# p2 = min(min(y_pred), min(y_test))
# plt.plot([p1, p2], [p1, p2], 'b-')
# plt.xlabel('True Values', fontsize=15)
# plt.ylabel('Predictions', fontsize=15)
# plt.axis('equal')
# plt.show()